In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re


from PIL import Image
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import pos_tag
import collections
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

import string
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.dummy import DummyClassifier

In [2]:
# pip install emoji --upgrade

In [3]:
import emoji

In [4]:
sw = list(stopwords.words('english'))

In [5]:
tweets = pd.read_csv('Data/Tweets.csv')

In [6]:
tweets.isna().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [7]:
tweets['negativereason_gold'].value_counts()

Customer Service Issue                      12
Late Flight                                  4
Cancelled Flight                             3
Can't Tell                                   3
Cancelled Flight\nCustomer Service Issue     2
Late Flight\nCancelled Flight                1
Late Flight\nFlight Attendant Complaints     1
Bad Flight                                   1
Customer Service Issue\nLost Luggage         1
Lost Luggage\nDamaged Luggage                1
Flight Attendant Complaints                  1
Late Flight\nLost Luggage                    1
Customer Service Issue\nCan't Tell           1
Name: negativereason_gold, dtype: int64

In [8]:
tweets['airline_sentiment_gold'].value_counts()

negative    32
positive     5
neutral      3
Name: airline_sentiment_gold, dtype: int64

In [9]:
tweets['retweet_count'].value_counts()

0     13873
1       640
2        66
3        22
4        17
5         5
7         3
6         3
22        2
8         1
32        1
9         1
31        1
18        1
15        1
28        1
44        1
11        1
Name: retweet_count, dtype: int64

In [10]:
tweets['tweet_created'].value_counts()

2015-02-24 09:54:34 -0800    5
2015-02-24 11:43:05 -0800    4
2015-02-23 15:25:46 -0800    3
2015-02-23 14:18:58 -0800    3
2015-02-23 10:58:58 -0800    3
                            ..
2015-02-21 18:14:18 -0800    1
2015-02-23 10:42:52 -0800    1
2015-02-22 13:06:59 -0800    1
2015-02-22 08:35:29 -0800    1
2015-02-22 20:08:49 -0800    1
Name: tweet_created, Length: 14247, dtype: int64

In [11]:
tweets['user_timezone'].value_counts()

Eastern Time (US & Canada)    3744
Central Time (US & Canada)    1931
Pacific Time (US & Canada)    1208
Quito                          738
Atlantic Time (Canada)         497
                              ... 
Warsaw                           1
Solomon Is.                      1
America/Detroit                  1
West Central Africa              1
Istanbul                         1
Name: user_timezone, Length: 85, dtype: int64

In [12]:
tweets['name'].value_counts()

JetBlueNews      63
kbosspotter      32
_mhertz          29
otisday          28
throthra         27
                 ..
zbarry1015        1
osman_khan        1
lalo_haros        1
philwa            1
travelmanphil     1
Name: name, Length: 7701, dtype: int64

In [13]:
tweets.drop('negativereason_gold', axis=1, inplace=True)
tweets.drop('airline_sentiment_gold', axis=1, inplace=True)
tweets.drop('retweet_count', axis=1, inplace=True)
tweets.drop('tweet_created', axis=1, inplace=True)
tweets.drop('user_timezone', axis=1, inplace=True)
tweets.drop('name', axis=1, inplace=True)

#current list of columns
list(tweets.columns)

['tweet_id',
 'airline_sentiment',
 'airline_sentiment_confidence',
 'negativereason',
 'negativereason_confidence',
 'airline',
 'text',
 'tweet_coord',
 'tweet_location']

In [14]:
tweets['airline_sentiment_confidence'].value_counts()

1.0000    10445
0.6667       71
0.6632       35
0.6559       30
0.6596       30
          ...  
0.3913        1
0.7273        1
0.6353        1
0.6260        1
0.3544        1
Name: airline_sentiment_confidence, Length: 1023, dtype: int64

In [15]:
tweets['negativereason'].value_counts()

Customer Service Issue         2910
Late Flight                    1665
Can't Tell                     1190
Cancelled Flight                847
Lost Luggage                    724
Bad Flight                      580
Flight Booking Problems         529
Flight Attendant Complaints     481
longlines                       178
Damaged Luggage                  74
Name: negativereason, dtype: int64

In [16]:
tweets['negativereason_confidence'].value_counts()

1.0000    3436
0.0000    1344
0.6667      62
0.6632      33
0.6596      29
          ... 
0.3386       1
0.3577       1
0.3249       1
0.3290       1
0.3255       1
Name: negativereason_confidence, Length: 1410, dtype: int64

In [17]:
tweets['airline'].value_counts()

United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: airline, dtype: int64

In [18]:
tweets['tweet_coord'].value_counts()

[0.0, 0.0]                      164
[40.64656067, -73.78334045]       6
[32.91792297, -97.00367737]       3
[40.64646912, -73.79133606]       3
[37.78618135, -122.45742542]      2
                               ... 
[37.77414482, -122.43628588]      1
[25.8058675, -80.1260541]         1
[36.11159761, -86.7844525]        1
[19.43706642, -99.07927123]       1
[34.02761838, -118.49646336]      1
Name: tweet_coord, Length: 832, dtype: int64

In [19]:
tweets['tweet_location'].value_counts()

Boston, MA                 157
New York, NY               156
Washington, DC             150
New York                   127
USA                        126
                          ... 
Raising Through 7            1
Orange County, USA           1
Rochester, Michigan          1
New York, NY | Brooklyn      1
Valrico, Florida             1
Name: tweet_location, Length: 3081, dtype: int64

In [20]:
# tweets.drop('tweet_id', axis=1, inplace=True)
# tweets['name'].value_counts()
# tweets['name'].value_counts()
# tweets['name'].value_counts()

In [21]:
tweets = tweets[tweets['airline_sentiment_confidence'] > 0.80]

In [22]:
tweets.shape

(10459, 9)

In [23]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10459 entries, 0 to 14638
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      10459 non-null  int64  
 1   airline_sentiment             10459 non-null  object 
 2   airline_sentiment_confidence  10459 non-null  float64
 3   negativereason                7392 non-null   object 
 4   negativereason_confidence     7396 non-null   float64
 5   airline                       10459 non-null  object 
 6   text                          10459 non-null  object 
 7   tweet_coord                   746 non-null    object 
 8   tweet_location                7007 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 817.1+ KB


In [24]:
#just in case
def labels_to_num(label):
    if label == 'negative':
        label = 0
    elif label == 'positive':
        label = 1
    else:
        label = 2
    return label

In [25]:
tweets['airline_sentiment_num'] = tweets['airline_sentiment'].map(labels_to_num)

In [26]:
tweets['airline_sentiment_num'].value_counts()

0    7392
2    1550
1    1517
Name: airline_sentiment_num, dtype: int64

In [27]:
##Translate nltk POS to wordnet tags for lemmatizer
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [28]:
# from nltk.stem import PorterStemmer

In [29]:
from nltk.tokenize import word_tokenize

In [30]:
lemmatizer = WordNetLemmatizer() #lemmatizing
#STOPWORDS

def process_text(text):
    doc = text.lower()
    doc = re.sub(r'@\w+', '', doc) # Remove @s
    doc = re.sub(r'#', '', doc) # Remove hashtags
    doc = re.sub(r':', ' ', emoji.demojize(doc)) # Turn emojis into words
    doc = re.sub(r'http\S+', '',doc) # Remove URLs
    doc = re.sub(r'\$\S+', 'dollar', doc) # Change dollar amounts to dollar
    doc = re.sub(r'[^a-z0-9\s]', '', doc) # Remove punctuation
    doc = re.sub(r'[0-9]+', 'number', doc) # Change number values to number
    
    doc = word_tokenize(doc)
#     doc = [word for word in doc if word not in sw]
   
    
    doc = pos_tag(doc)

    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]

    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc] 

    return ' '.join(doc)

In [31]:
tweets['processed_text'] = tweets['text'].map(process_text)

In [32]:
tweets['text'][0]

'@VirginAmerica What @dhepburn said.'

In [33]:
tweets['processed_text'][0]

'what say'

In [34]:
dummied_airlines = pd.get_dummies(tweets['airline'])

In [35]:
tweets = tweets.join(dummied_airlines)

In [36]:
tweets.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline', 'text',
       'tweet_coord', 'tweet_location', 'airline_sentiment_num',
       'processed_text', 'American', 'Delta', 'Southwest', 'US Airways',
       'United', 'Virgin America'],
      dtype='object')

## Vectorizing

In [37]:
X = tweets.drop(columns=['tweet_id', 'airline_sentiment_confidence', 'airline',
       'negativereason', 'negativereason_confidence', 'text', 'tweet_coord', 'tweet_location', 'airline_sentiment_num', 'airline_sentiment'], axis =1)
y = tweets['airline_sentiment']

In [38]:
##running our train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.20)

In [39]:
#try different parameters
vectorizer = TfidfVectorizer(input='content')

In [40]:
sparse_out = vectorizer.fit_transform(X_train['processed_text'])
tfidf_df_train = pd.DataFrame(data = sparse_out.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_train.index)


X_train.drop(['processed_text'], axis=1, inplace = True)
X_train_vec = X_train.join(tfidf_df_train)

C:\Users\Admin\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [41]:
sparse_out1 = vectorizer.transform(X_test['processed_text'])
tfidf_df_test = pd.DataFrame(data = sparse_out1.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_test.index)

X_test.drop(['processed_text'], axis=1, inplace = True)
X_test_vec = X_test.join(tfidf_df_test)
X_test_vec = X_test_vec.fillna(value=0)

C:\Users\Admin\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [42]:
#smoteN

In [43]:
tweets.airline_sentiment.value_counts()

negative    7392
neutral     1550
positive    1517
Name: airline_sentiment, dtype: int64

In [44]:
X_train_vec.shape

(8367, 8782)

In [45]:
X_test_vec.shape

(2092, 8782)

### Dummy classifier

In [56]:
##checking our dummy/base model score
dc = DummyClassifier(strategy='stratified')
dc.fit(X_train_vec, y_train)

print(cross_val_score(dc, X_train_vec, y_train).mean())

0.548342249273549


### Support Vector Classifier

In [46]:
svc = svm.SVC() #Grid search ()
svc.fit(X_train_vec, y_train)

print(cross_val_score(svc, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed: 11.0min remaining: 16.5min
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed: 11.0min finished


0.8355443579630378


In [57]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix()

In [79]:
from sklearn.linear_model import LogisticRegression

In [80]:
lr = LogisticRegression() #max iterations-change; l2 or l1 - Grid search (l1, l2, C=[.1, 1, 10])  # Suppose .1 is best, then for Round #2 C=[.03, .1, .3
lr.fit(X_train_vec, y_train)
print(cross_val_score(lr, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

C:\Users\Admin\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:   37.0s remaining:   55.6s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:   38.4s finished


0.8656628824802667


In [81]:
pred= lr.predict(X_test_vec)

In [82]:
pred_train= lr.predict(X_train_vec)

In [83]:
accuracy_score(y_train, pred_train)

0.925660332257679

In [84]:
accuracy_score(y_test, pred)

0.880019120458891

### Naive Bayes

In [47]:
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
print(cross_val_score(nb, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:    2.3s remaining:    3.4s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:    2.3s finished


0.7422002126685621


### Required Configurations and Python Libraries

In [46]:
# pip install tweepy

In [47]:
import tweepy as tw
import json
import time
import datetime
import os
import csv
import pickle

In [48]:
Twitter={}
Twitter['Consumer Key'] = 'iAimqbAdWiBrUfXe0Xmuhlyps'
Twitter['Consumer Secret'] = 'YWUyAsX8sA2VuJCwQv4x3MUhAGmm48VlV0ZTx2yWNWqlGCP9Kj'
Twitter['Access Token'] = '54188780-kRmlqKRHleSb1WBBTfrj4CR57wLxJYuU5qH9piYlq'
Twitter['Access Token Secret'] = 'JXzf6VKWIFhVhAuIYWWQX04jCaHgfgjia77ARpK7v1Z3p'

In [49]:
!pip install twitter

In [50]:
import twitter

auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

In [51]:
airlines_list = tweets.columns[-6:].tolist()
airlines_dict = {i: '#'+i.replace(' ', '') for i in airlines_list }

In [52]:
airlines_dict

{'American': '#American',
 'Delta': '#Delta',
 'Southwest': '#Southwest',
 'US Airways': '#USAirways',
 'United': '#United',
 'Virgin America': '#VirginAmerica'}

In [53]:
airlines_dict['American'] = '#AmericanAirlines'
airlines_dict['United'] = '#UnitedAirlines'
airlines_dict['Southwest'] = '#SouthwestAirlines'
airlines_dict['Delta'] = '#DeltaAirlines'

In [92]:
def get_tweets(hashtag, number):
    list_of_tweets =[]
    list_of_dirty_tweets =[]
    search_results = twitter_api.search.tweets(q=hashtag, count=number)
    print(hashtag + ':' + str(len(search_results['statuses'])) + "\n")
    for num in range(len(search_results['statuses'])):
        list_of_tweets.append(process_text(search_results['statuses'][num]['text']))
        list_of_dirty_tweets.append(search_results['statuses'][num]['text'])
        
    return list_of_tweets, list_of_dirty_tweets

In [73]:
airlines_dict

{'American': '#AmericanAirlines',
 'Delta': '#DeltaAirlines',
 'Southwest': '#SouthwestAirlines',
 'US Airways': '#USAirways',
 'United': '#UnitedAirlines',
 'Virgin America': '#VirginAmerica'}

In [99]:
X_real_by_airline = {}
for column_name, hashtag  in airlines_dict.items():
    X_real_one_airline = pd.DataFrame(get_tweets(hashtag, 1000)[0], columns=['processed_text'])
    for other_col_name in airlines_dict.keys():
        if other_col_name == column_name:
            X_real_one_airline[column_name] = 1
        else:
            X_real_one_airline[other_col_name] = 0
            
    X_real_by_airline[column_name] = X_real_one_airline
    
    
#X_real = pd.DataFrame(get_tweets('#DeltaAirlines', 7), columns=['processed_text'])

#AmericanAirlines:100

#DeltaAirlines:100

#SouthwestAirlines:100

#USAirways:4

#UnitedAirlines:100

#VirginAmerica:11



In [100]:
sparse_out2 = vectorizer.transform(X_real_by_airline['Delta']['processed_text'])
tfidf_df_real = pd.DataFrame(data = sparse_out2.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_real_by_airline['Delta'].index)


X_real_by_airline['Delta'].drop(['processed_text'], axis=1, inplace = True)
X_real_vec = X_real_by_airline['Delta'].join(tfidf_df_real)

In [102]:
preds = lr.predict(X_real_vec)

In [103]:
pd.concat(pd.DataFrame(preds), pd.DataFrame()

,0
0,neutral
1,neutral
2,negative
3,neutral
4,neutral
...,...
95,negative
96,negative
97,negative
98,neutral


In [ ]:
q = '#Southwest' 

number = 10

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets

search_results = twitter_api.search.tweets(q=q, count=number)


In [ ]:
search_results

In [ ]:
process_text(search_results['statuses'][9]['text'])